# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [ ]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
!conda build ../../bin/conda/libbasic -c statiskit -c conda-forge
!conda install -y libbasic --use-local -c statiskit -c conda-forge

BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py36_0 conda-forge
    libdev:          1.0.0-py36_0       statiskit  
    ncurses:         5.9-10             conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py36_0      conda-forge
    pip:             9.0.1-py36_0       conda-forge
    python:          3.6.2-0            conda-forge
    python-scons:    3.0.0-py36_0       statiskit  
    pyyaml:          3.12-py36_1        conda-forge
    readline:        6.2-0              conda-forge
    setuptools:      36.3.0-py36_0      conda-forge
    six:             1.10.0-py36_1      conda-forge
    sqlite:          3.13.0-1           conda-forge
    tk:              8.5.19-2           conda-forge
    wheel:           0.29.0-py36_0      conda-forge
    xz:              

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [ ]:
%%time
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [ ]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = '../../src/py/_basic.cpp',
                             decorator = '../../src/py/basic/_basic.py',
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time
wrappers.write()

Here is the list of the generated wrappers (new files).

In [ ]:
!git status

And here, we present the wrappers for the `BinomialDistribution` class.

In [ ]:
!pygmentize ../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
!conda build -q ../../bin/conda/python-basic -c statiskit -c conda-forge
!conda install -y -q python-basic --use-local -c statiskit --force

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')